##Exercise Overview
In this exercise we will play with Spark [Datasets & Dataframes](https://spark.apache.org/docs/latest/sql-programming-guide.html#datasets-and-dataframes), some [Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html#sql), and build a couple of binary classifiaction models using [Spark ML](https://spark.apache.org/docs/latest/ml-guide.html) (with some [MLlib](https://spark.apache.org/mllib/) too). 
<br><br>
The set up and approach will not be too dissimilar to the standard type of approach you might do in [Sklearn](http://scikit-learn.org/stable/index.html). Spark has matured to the stage now where for 90% of what you need to do (when analysing tabular data) should be possible with Spark dataframes, SQL, and ML libraries. This is where this exercise is mainly trying to focus.  
<br>
Feel free to adapt this exercise to play with other datasets readily availabe in the Databricks enviornment (they are listed in a cell below). 
#####Getting Started
To get started you will need to create and attach a databricks spark cluster to this notebook. This notebook was developed on a cluster created with: 
- Databricks Runtime Version 4.0 (includes Apache Spark 2.3.0, Scala 2.11)
- Python Version 3

#####Links & References

Some useful links and references of sources used in creating this exercise:

**Note**: Right click and open as new tab!
<br>
1. [Latest Spark Docs](https://spark.apache.org/docs/latest/index.html)
1. [Databricks Homepage](https://databricks.com/)
1. [Databricks Community Edition FAQ](https://databricks.com/product/faq/community-edition)
1. [Databricks Self Paced Training](https://databricks.com/training-overview/training-self-paced)
1. [Databricks Notebook Guide](https://docs.databricks.com/user-guide/notebooks/index.html)
1. [Databricks Binary Classification Tutorial](https://docs.databricks.com/spark/latest/mllib/binary-classification-mllib-pipelines.html#binary-classification)

####Get Data

Here we will pull in some sample data that is already pre-loaded onto all databricks clusters.

Feel free to adapt this notebook later to play around with a different dataset if you like (all available are listed in a cell below).

In [3]:
# display datasets already in databricks
display(dbutils.fs.ls("/databricks-datasets"))

path,name,size
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-datasets/COVID/,COVID/,0
dbfs:/databricks-datasets/README.md,README.md,976
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359
dbfs:/databricks-datasets/adult/,adult/,0
dbfs:/databricks-datasets/airlines/,airlines/,0
dbfs:/databricks-datasets/amazon/,amazon/,0
dbfs:/databricks-datasets/asa/,asa/,0
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0


Lets take a look at the '**adult**' dataset on the filesystem. This is the typical US Census data you often see online in tutorials. [Here](https://archive.ics.uci.edu/ml/datasets/adult) is the same data in the UCI repository.

_As an aside: [here](https://github.com/GoogleCloudPlatform/cloudml-samples/tree/master/census) this same dataset is used as a quickstart example for Google CLoud ML & Tensorflow Estimator API (in case youd be interested in playing with tensorflow on the same dataset as here)._

In [5]:
%fs ls dbfs:/databricks-datasets/adult/adult.data

path,name,size
dbfs:/databricks-datasets/adult/adult.data,adult.data,3974305


**Note**: Above  %fs is just some file system cell magic that is specific to databricks. More info [here](https://docs.databricks.com/user-guide/notebooks/index.html#mix-languages).

####Spark SQL
Below we will use Spark SQL to load in the data and then register it as a Dataframe aswell. So the end result will be a Spark SQL table called _adult_ and a Spark Dataframe called _df_adult_. 
<br><br>
This is an example of the flexibility in Spark in that you could do lots of you ETL and data wrangling using either Spark SQL or Dataframes and pyspark. Most of the time it's a case of using whatever you are most comfortable with.
<br><br>
When you get more advanced then you might looking the pro's and con's of each and when you might favour one or the other (or operating direclty on RDD's), [here](https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html) is a good article on the issues. For now, no need to overthink it!

In [8]:
%sql 
-- drop the table if it already exists
DROP TABLE IF EXISTS adult

In [9]:
%sql
-- create a new table in Spark SQL from the datasets already loaded in the underlying filesystem.
-- In the real world you might be pointing at a file on HDFS or a hive table etc. 
CREATE TABLE adult (
  age DOUBLE,
  workclass STRING,
  fnlwgt DOUBLE,
  education STRING,
  education_num DOUBLE,
  marital_status STRING,
  occupation STRING,
  relationship STRING,
  race STRING,
  sex STRING,
  capital_gain DOUBLE,
  capital_loss DOUBLE,
  hours_per_week DOUBLE,
  native_country STRING,
  income STRING)
USING com.databricks.spark.csv
OPTIONS (path "/databricks-datasets/adult/adult.data", header "true")

In [10]:
# look at the data
#spark.sql("SELECT * FROM adult LIMIT 5").show() 
# this will look prettier in Databricks if you use display() instead
display(spark.sql("SELECT * FROM adult LIMIT 5"))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
37.0,Private,284582.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K


If you are more comfortable with SQL then as you can see below, its very easy to just get going with writing standard SQL type code to analyse your data, do data wrangling and create new dataframes.

In [12]:
# Lets get some summary marital status rates by occupation
result = spark.sql(
  """
  SELECT 
    occupation,
    SUM(1) as n,
    ROUND(AVG(if(LTRIM(marital_status) LIKE 'Married-%',1,0)),2) as married_rate,
    ROUND(AVG(if(lower(marital_status) LIKE '%widow%',1,0)),2) as widow_rate,
    ROUND(AVG(if(LTRIM(marital_status) = 'Divorced',1,0)),2) as divorce_rate,
    ROUND(AVG(if(LTRIM(marital_status) = 'Separated',1,0)),2) as separated_rate,
    ROUND(AVG(if(LTRIM(marital_status) = 'Never-married',1,0)),2) as bachelor_rate
  FROM 
    adult 
  GROUP BY 1
  ORDER BY n DESC
  """)
display(result)

occupation,n,married_rate,widow_rate,divorce_rate,separated_rate,bachelor_rate
Prof-specialty,4140,0.53,0.02,0.13,0.02,0.3
Craft-repair,4099,0.64,0.01,0.11,0.03,0.21
Exec-managerial,4066,0.61,0.02,0.15,0.02,0.2
Adm-clerical,3769,0.28,0.04,0.22,0.04,0.42
Sales,3650,0.47,0.03,0.12,0.03,0.36
Other-service,3295,0.24,0.05,0.15,0.06,0.5
Machine-op-inspct,2002,0.51,0.03,0.14,0.04,0.29
?,1843,0.36,0.08,0.1,0.04,0.42
Transport-moving,1597,0.63,0.02,0.11,0.02,0.21
Handlers-cleaners,1370,0.36,0.01,0.09,0.03,0.51


You can easily register dataframes as a table for Spark SQL too. So this way you can easily move between Dataframes and Spark SQL for whatever reason.

In [14]:
# register the df we just made as a table for spark sql
sqlContext.registerDataFrameAsTable(result, "result")
spark.sql("SELECT * FROM result").show(5)

+----------------+----+------------+----------+------------+--------------+-------------+
 occupation| n|married_rate|widow_rate|divorce_rate|separated_rate|bachelor_rate|
+----------------+----+------------+----------+------------+--------------+-------------+
 Prof-specialty|4140| 0.53| 0.02| 0.13| 0.02| 0.3|
 Craft-repair|4099| 0.64| 0.01| 0.11| 0.03| 0.21|
 Exec-managerial|4066| 0.61| 0.02| 0.15| 0.02| 0.2|
 Adm-clerical|3769| 0.28| 0.04| 0.22| 0.04| 0.42|
 Sales|3650| 0.47| 0.03| 0.12| 0.03| 0.36|
+----------------+----+------------+----------+------------+--------------+-------------+
only showing top 5 rows

####<span style="color:darkblue">Question 1</span>

1. Write some spark sql to get the top 'bachelor_rate' by 'education' group?

In [16]:
### Question 1.1 Answer ###
result = spark.sql(
  """
  SELECT education, count(education) as n,
  ROUND(AVG(if(LTRIM(marital_status) = 'Never-married',1,0)),2) as bachelor_rate
    FROM adult 
      GROUP BY education   
        ORDER BY bachelor_rate desc
          limit 1
  """)
display(result)

education,n,bachelor_rate
12th,433,0.54


In [17]:
sqlContext.registerDataFrameAsTable(result, "result")
spark.sql("SELECT * FROM result").show()

+---------+---+-------------+
education| n|bachelor_rate|
+---------+---+-------------+
 12th|433| 0.54|
+---------+---+-------------+

####Spark DataFrames
Below we will create our DataFrame from the SQL table and do some similar analysis as we did with Spark SQL but using the DataFrames API.

In [19]:
# register a df from the sql df
df_adult = spark.table("adult")
cols = df_adult.columns # this will be used much later in the notebook, ignore for now

In [20]:
# look at df schema
df_adult.printSchema()

root
-- age: double (nullable = true)
-- workclass: string (nullable = true)
-- fnlwgt: double (nullable = true)
-- education: string (nullable = true)
-- education_num: double (nullable = true)
-- marital_status: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- sex: string (nullable = true)
-- capital_gain: double (nullable = true)
-- capital_loss: double (nullable = true)
-- hours_per_week: double (nullable = true)
-- native_country: string (nullable = true)
-- income: string (nullable = true)

In [21]:
df_adult.show(5)

+----+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
 age| workclass| fnlwgt| education|education_num| marital_status| occupation| relationship| race| sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+----+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
50.0| Self-emp-not-inc| 83311.0| Bachelors| 13.0| Married-civ-spouse| Exec-managerial| Husband| White| Male| 0.0| 0.0| 13.0| United-States| <=50K|
38.0| Private|215646.0| HS-grad| 9.0| Divorced| Handlers-cleaners| Not-in-family| White| Male| 0.0| 0.0| 40.0| United-States| <=50K|
53.0| Private|234721.0| 11th| 7.0| Married-civ-spouse| Handlers-cleaners| Husband| Black| Male| 0.0| 0.0| 40.0| United-States| <=50K|
28.0| Private|338409.0| Bachelors| 13.0| Married-civ-spouse| Prof-specialty| Wife| Black| Female| 0.0| 0.0| 40.0| Cuba| <=50K|
37.0| Private|284582.0| Masters| 14.0| Married-civ-spouse| Exec-managerial| Wife| White| Female| 0.0| 0.0| 40.0| United-States| <=50K|
+----+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
only showing top 5 rows

Below we will do a similar calculation to what we did above but using the DataFrames API

In [23]:
# import what we will need
from pyspark.sql.functions import when, col, mean, desc, round

# wrangle the data a bit
df_result = df_adult.select(
  df_adult['occupation'], 
  # create a 1/0 type col on the fly
  when( col('marital_status') == ' Divorced' , 1 ).otherwise(0).alias('is_divorced')
)
# do grouping (and a round)
df_result = df_result.groupBy('occupation').agg(round(mean('is_divorced'),2).alias('divorced_rate'))
# do ordering
df_result = df_result.orderBy(desc('divorced_rate'))
# show results
df_result.show(5)

+----------------+-------------+
 occupation|divorced_rate|
+----------------+-------------+
 Adm-clerical| 0.22|
 Priv-house-serv| 0.19|
 Other-service| 0.15|
 Tech-support| 0.15|
 Exec-managerial| 0.15|
+----------------+-------------+
only showing top 5 rows

As you can see the dataframes api is a bit more verbose then just expressing what you want to do in standard SQL.<br><br>But some prefer it and might be more used to it, and there could be cases where expressing what you need to do might just be better using the DataFrame API if it is too complicated for a simple SQL expression for example of maybe involves recursion of some type.

####<span style="color:darkblue">Question 2</span>
1. Write some pyspark to get the top 'bachelor_rate' by 'education' group using DataFrame operations?

In [26]:
### Question 2.1 Answer ###
from pyspark.sql.functions import when, col, mean, desc, round
# wrangle the data a bit
# wrangle the data a bit
df_result = df_adult.select(
  df_adult['education'], 
  # create a 1/0 type col on the fly
  when( col('marital_status') == ' Never-married' , 1 ).otherwise(0).alias('some_sta')
)
# do grouping (and a round)
df_result = df_result.groupBy('education').agg(round(mean('some_sta'),2).alias('bachelor_rate'))
# do ordering
df_result = df_result.orderBy(desc('bachelor_rate'))
# show results
df_result.show(1)

+---------+-------------+
education|bachelor_rate|
+---------+-------------+
 12th| 0.54|
+---------+-------------+
only showing top 1 row

####Explore & Visualize Data
It's very easy to [collect()](https://spark.apache.org/docs/latest/rdd-programming-guide.html#printing-elements-of-an-rdd) your Spark DataFrame data into a Pandas df and then continue to analyse or plot as you might normally.
<br><br>
Obviously if you try to collect() a huge DataFrame then you will run into issues, so usually you would only collect aggregated or sampled data into a Pandas df.

In [28]:
import pandas as pd

# do some analysis
result = spark.sql(
  """
  SELECT 
    occupation,
    AVG(IF(income = ' >50K',1,0)) as plus_50k
      FROM 
      adult 
        GROUP BY 1
          ORDER BY 2 DESC
  """)

# collect results into a pandas df
df_pandas = pd.DataFrame(
  result.collect(),
  columns=result.schema.names
)

# look at df
print(df_pandas.head())

occupation plus_50k
0 Exec-managerial 0.484014
1 Prof-specialty 0.449034
2 Protective-serv 0.325116
3 Tech-support 0.304957
4 Sales 0.269315

In [29]:
print(df_pandas.describe())

plus_50k
count 15.000000
mean 0.197357
std 0.143993
min 0.006711
25% 0.107373
50% 0.134518
75% 0.287136
max 0.484014

In [30]:
print(df_pandas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
occupation 15 non-null object
plus_50k 15 non-null float64
dtypes: float64(1), object(1)
memory usage: 320.0+ bytes
None

Here we will just do some very basic plotting to show how you might collect what you are interested in into a Pandas DF and then just plot any way you normally would.

For simplicity we are going to use the plotting functionality built into pandas (you could make this a pretty as you want).

In [32]:
import matplotlib.pyplot as plt

# i like ggplot style
plt.style.use('ggplot')

# get simple plot on the pandas data
myplot = df_pandas.plot(kind='barh', x='occupation', y='plus_50k')

# display the plot (note - display() is a databricks function - 
# more info on plotting in Databricks is here: https://docs.databricks.com/user-guide/visualizations/matplotlib-and-ggplot.html)
display(myplot.figure)

You can also easily get summary stats on a Spark DataFrame like below. [Here](https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html) is a nice blog post that has more examples.<br><br>So this is an example of why you might want to move from Spark SQL into DataFrames API as being able to just call describe() on the Spark DF is easier then trying to do the equivilant in Spark SQL.

In [34]:
# describe df
df_adult.select(df_adult['age'],df_adult['education_num']).describe().show()

+-------+------------------+------------------+
summary| age| education_num|
+-------+------------------+------------------+
 count| 32560| 32560|
 mean|38.581633906633904| 10.08058968058968|
 stddev|13.640641827464002|2.5727089681052058|
 min| 17.0| 1.0|
 max| 90.0| 16.0|
+-------+------------------+------------------+

### ML Pipeline - Logistic Regression vs Random Forest

Below we will create two [Spark ML Pipelines](https://spark.apache.org/docs/latest/ml-pipeline.html) - one that fits a logistic regression and one that fits a random forest. We will then compare the performance of each.

**Note**: A lot of the code below is adapted from [this example](https://docs.databricks.com/spark/latest/mllib/binary-classification-mllib-pipelines.html).

In [36]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

categoricalColumns = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country"]
stages = [] # stages in our Pipeline

for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    # encoder = OneHotEncoderEstimator(inputCol=categoricalCol + "Index", outputCol=categoricalCol + "classVec")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

In [37]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol="income", outputCol="label")
stages += [label_stringIdx]

In [38]:
# Transform all features into a vector using VectorAssembler
numericCols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [39]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(df_adult)
dataset = pipelineModel.transform(df_adult)
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)

In [40]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)

In [41]:
from pyspark.sql.functions import avg

# get the rate of the positive outcome from the training data to use as a threshold in the model
training_data_positive_rate = trainingData.select(avg(trainingData['label'])).collect()[0][0] 

print("Positive rate in the training data is {}".format(training_data_positive_rate))

Positive rate in the training data is 0.23934842580023646

####Logistic Regression - Train

In [43]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# set threshold for the probability above which to predict a 1
lr.setThreshold(training_data_positive_rate)
# lr.setThreshold(0.5) # could use this if knew you had balanced data

# Train model with Training Data
lrModel = lr.fit(trainingData)

# get training summary used for eval metrics and other params
lrTrainingSummary = lrModel.summary

# Find the best model threshold if you would like to use that instead of the empirical positve rate
fMeasure = lrTrainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
lrBestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
  
print("Best threshold based on model performance on training data is {}".format(lrBestThreshold))

Best threshold based on model performance on training data is 0.3334162417235758

####GBM - Train

####<span style="color:darkblue">Question 3</span>
1. Train a GBTClassifier on the training data, call the trained model 'gbModel'

In [46]:
### Question 3.1 Answer ###
from pyspark.ml.classification import GBTClassifier

# Create initial LogisticRegression model
gb = GBTClassifier(maxIter=10)

# Train model with Training Data
gbModel = gb.fit(trainingData)

####Logistic Regression - Predict

In [48]:
# make predictions on test data
lrPredictions = lrModel.transform(testData)

# display predictions
#display(lrPredictions.select("label", "prediction", "probability"))
#display(lrPredictions)

###GBM - Predict

####<span style="color:darkblue">Question 4</span>
1. Get predictions on the test data for your GBTClassifier. Call the predictions df 'gbPredictions'.

In [51]:
### Question 4.1 Answer ###

# make predictions on test data
gbPredictions = gbModel.transform(testData)

#display(gbPredictions)

####Logistic Regression - Evaluate

####<span style="color:darkblue">Question 5</span>
1. Complete the print_performance_metrics() function below to also include measures of F1, Precision, Recall, False Positive Rate and True Positive Rate.

In [54]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics

def print_performance_metrics(predictions):
  # Evaluate model
  evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
  auc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
  aupr = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})
  print("auc = {}".format(auc))
  print("aupr = {}".format(aupr))

  # get rdd of predictions and labels for mllib eval metrics
  predictionAndLabels = predictions.select("prediction","label").rdd

  # Instantiate metrics objects
  binary_metrics = BinaryClassificationMetrics(predictionAndLabels)
  multi_metrics = MulticlassMetrics(predictionAndLabels)

  # Area under precision-recall curve
  print("Area under PR = {}".format(binary_metrics.areaUnderPR))
  # Area under ROC curve
  print("Area under ROC = {}".format(binary_metrics.areaUnderROC))
  # Accuracy
  print("Accuracy = {}".format(multi_metrics.accuracy))
  # Confusion Matrix
  print(multi_metrics.confusionMatrix())
  
print_performance_metrics(lrPredictions)

auc = 0.9032867661805299
aupr = 0.7627830907418989
Area under PR = 0.5366100314564946
Area under ROC = 0.8159794860040686
Accuracy = 0.80067880283863
DenseMatrix([[5776., 1572.],
 [ 366., 2009.]])

####GBM - Evaluate

In [56]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics

def print_performance_metrics(predictions):
  # Evaluate model
  evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
  auc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
  aupr = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})
  print("auc = {}".format(auc))
  print("aupr = {}".format(aupr))

  # get rdd of predictions and labels for mllib eval metrics
  predictionAndLabels = predictions.select("prediction","label").rdd

  # Instantiate metrics objects
  binary_metrics = BinaryClassificationMetrics(predictionAndLabels)
  multi_metrics = MulticlassMetrics(predictionAndLabels)
  
  ### Question 5.1 Answer ###
  
  # F1
  print("F1 = {}".format(multi_metrics.fMeasure()))
  # Precision
  print("Precision = {}".format(multi_metrics.precision()))
  # Recall
  print("Recall = {}".format(multi_metrics.recall()))
  # FPR
  print("FPR = {}".format(multi_metrics.weightedFalsePositiveRate))
  # TPR
  print("TPR = {}".format(multi_metrics.weightedTruePositiveRate))
  

In [57]:
print_performance_metrics(gbPredictions)

auc = 0.903386757585308
aupr = 0.7738361178782271
F1 = 0.8510747711611643
Precision = 0.8510747711611643
Recall = 0.8510747711611643
FPR = 0.3422151315886347
TPR = 0.8510747711611643

## Cross Validation

For each model you can run the below comand to see its params and a brief explanation of each.

In [60]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal with 1 for binomial regression, or the number oflasses for multinomial regression. (undefined)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0)
standardization: whether to standardize the training features before fitting the model. (default: True)
threshold: Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p]. (default: 0.5, current: 0.23934842580023646)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold. (undefined)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
upperBoundsOnCoefficients: The upper bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
upperBoundsOnIntercepts: The upper bounds on intercepts if fitting under bound constrained optimization. The bound vector size must be equal with 1 for binomial regression, or the number of classes for multinomial regression. (undefined)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

In [61]:
print(gb.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto' (default: all)
featuresCol: features column name. (default: features)
labelCol: label column name. (default: label)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic (default: logistic)
maxBins: Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)
maxIter: max number of iterations (>= 0). (default: 20, current: 10)
maxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size. (default: 256)
minInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)
minInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1. (default: 1)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: 3504127614838123891)
stepSize: Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator. (default: 0.1)
subsamplingRate: Fraction of the training data used for learning each decision tree, in range (0, 1]. (default: 1.0)

####Logisitic Regression - Param Grid

In [63]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
lrParamGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [2, 5])
             .build())

####GBM - Param Grid

####<span style="color:darkblue">Question 6</span>
1. Build out a param grid for the gb model, call it 'gbParamGrid'.

In [66]:
### Question 6.1 Answer ###

# Create ParamGrid for Cross Validation
gbParamGrid = (ParamGridBuilder()
             .addGrid(gb.maxDepth, [5,12])
             .addGrid(gb.maxBins, [32,64])
             .addGrid(gb.maxIter, [2, 5])
             .build())

####Logistic Regression - Perform Cross Validation

In [68]:
# set up an evaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

# Create CrossValidator
lrCv = CrossValidator(estimator=lr, estimatorParamMaps=lrParamGrid, evaluator=evaluator, numFolds=2)

# Run cross validations
lrCvModel = lrCv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [69]:
# below approach to getting at the best params from the best cv model taken from:
# https://stackoverflow.com/a/46353730/1919374

# look at best params from the CV
print(lrCvModel.bestModel._java_obj.getRegParam())
print(lrCvModel.bestModel._java_obj.getElasticNetParam())
print(lrCvModel.bestModel._java_obj.getMaxIter())

0.01
0.0
5

####GBM - Perform Cross Validation

####<span style="color:darkblue">Question 7</span>
1. Perform cross validation of params on your 'gb' model.
1. Print out the best params you found.

In [72]:
### Question 7.1 Answer ###

# Create CrossValidator
gbCv = CrossValidator(estimator=gb, estimatorParamMaps=gbParamGrid, evaluator=evaluator, numFolds=2)

# Run cross validations
gbCvModel = gbCv.fit(trainingData)

# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [73]:
### Question 7.2 Answer ###  
# https://stackoverflow.com/questions/36697304/how-to-extract-model-hyper-parameters-from-spark-ml-in-pyspark
# look at best params from the CV
print(gbCvModel.bestModel._java_obj.extractParamMap())

{
	GBTClassifier_89c9c1c17dc6-cacheNodeIds: false,
	GBTClassifier_89c9c1c17dc6-checkpointInterval: 10,
	GBTClassifier_89c9c1c17dc6-featureSubsetStrategy: all,
	GBTClassifier_89c9c1c17dc6-featuresCol: features,
	GBTClassifier_89c9c1c17dc6-impurity: gini,
	GBTClassifier_89c9c1c17dc6-labelCol: label,
	GBTClassifier_89c9c1c17dc6-lossType: logistic,
	GBTClassifier_89c9c1c17dc6-maxBins: 64,
	GBTClassifier_89c9c1c17dc6-maxDepth: 5,
	GBTClassifier_89c9c1c17dc6-maxIter: 5,
	GBTClassifier_89c9c1c17dc6-maxMemoryInMB: 256,
	GBTClassifier_89c9c1c17dc6-minInfoGain: 0.0,
	GBTClassifier_89c9c1c17dc6-minInstancesPerNode: 1,
	GBTClassifier_89c9c1c17dc6-predictionCol: prediction,
	GBTClassifier_89c9c1c17dc6-probabilityCol: probability,
	GBTClassifier_89c9c1c17dc6-rawPredictionCol: rawPrediction,
	GBTClassifier_89c9c1c17dc6-seed: 3504127614838123891,
	GBTClassifier_89c9c1c17dc6-stepSize: 0.1,
	GBTClassifier_89c9c1c17dc6-subsamplingRate: 1.0,
	GBTClassifier_89c9c1c17dc6-validationTol: 0.01
}

####Logistic Regression - CV Model Predict

In [75]:
# Use test set to measure the accuracy of our model on new data
lrCvPredictions = lrCvModel.transform(testData)

#display(lrCvPredictions)

####GBM - CV Model Predict

In [77]:
gbCvPredictions = gbCvModel.transform(testData)

#display(gbCvPredictions)

####Logistic Regression - CV Model Evaluate

In [79]:
print_performance_metrics(lrCvPredictions)

auc = 0.8857251239148407
aupr = 0.7241684115655654
F1 = 0.773423840378484
Precision = 0.773423840378484
Recall = 0.773423840378484
FPR = 0.17667422000201208
TPR = 0.773423840378484

####GBM - CV Model Evaluate

In [81]:
print_performance_metrics(gbCvPredictions)

auc = 0.9032570552674553
aupr = 0.7814452781776516
F1 = 0.8512804689910521
Precision = 0.8512804689910521
Recall = 0.8512804689910521
FPR = 0.3498425983209092
TPR = 0.8512804689910523

####Logistic Regression - Model Explore

In [83]:
print('Model Intercept: ', lrCvModel.bestModel.intercept)

Model Intercept: -1.247913441799743

In [84]:
lrWeights = lrCvModel.bestModel.coefficients
lrWeights = [(float(w),) for w in lrWeights]  # convert numpy type to float, and to tuple
lrWeightsDF = sqlContext.createDataFrame(lrWeights, ["Feature Weight"])
display(lrWeightsDF)

Feature Weight
-0.22413336981436774
-0.3455553822296018
-0.13203533849479424
-0.4680986801474529
-0.24553588692884637
0.43228384563178
0.4075811047761166
-1.159748876366615
-0.4380095204656964
-0.19291567917419333


### Feature Importance

####<span style="color:darkblue">Question 8</span>
1. Print out a table of feature_name and feature_coefficient from the Logistic Regression model.
<br><br>
Hint: Adapt the code from here: https://stackoverflow.com/questions/42935914/how-to-map-features-from-the-output-of-a-vectorassembler-back-to-the-column-name

In [87]:
### Question 8.1 Answer ###

# from: https://stackoverflow.com/questions/42935914/how-to-map-features-from-the-output-of-a-vectorassembler-back-to-the-column-name

# fill in here
transformed = lrModel.transform(trainingData)
from itertools import chain

attrs = sorted(
(attr["idx"], attr["name"]) for attr in (chain(*transformed
.schema[lrModel.summary.featuresCol]
.metadata["ml_attr"]["attrs"].values())))
hold1=[(name, lrModel.coefficients[idx]) for idx, name in attrs]
hold1

Out[47]: [('workclassclassVec_ Private', -0.4216721741208325),
 ('workclassclassVec_ Self-emp-not-inc', -0.9311936077700329),
 ('workclassclassVec_ Local-gov', -0.6495180206787059),
 ('workclassclassVec_ ?', -0.6582722875549999),
 ('workclassclassVec_ State-gov', -0.7243157913223663),
 ('workclassclassVec_ Self-emp-inc', -0.40662624868197095),
 ('workclassclassVec_ Federal-gov', 0.08202162162712846),
 ('workclassclassVec_ Without-pay', -3.976520110093976),
 ('educationclassVec_ HS-grad', -0.717652729096384),
 ('educationclassVec_ Some-college', -0.3212675362252498),
 ('educationclassVec_ Bachelors', 0.6907649802430208),
 ('educationclassVec_ Masters', 1.1234779832272794),
 ('educationclassVec_ Assoc-voc', -0.11507831533890156),
 ('educationclassVec_ 11th', -1.584501434404926),
 ('educationclassVec_ Assoc-acdm', 0.02205028927123226),
 ('educationclassVec_ 10th', -1.8608951207374278),
 ('educationclassVec_ 7th-8th', -2.6174707319081785),
 ('educationclassVec_ Prof-school', 1.6150252850157616),
 ('educationclassVec_ 9th', -2.290393709859191),
 ('educationclassVec_ 12th', -1.1642948236279627),
 ('educationclassVec_ Doctorate', 1.9144132054839458),
 ('educationclassVec_ 5th-6th', -2.1722164825203953),
 ('educationclassVec_ 1st-4th', -2.417198471313863),
 ('marital_statusclassVec_ Married-civ-spouse', 0.5332151653224282),
 ('marital_statusclassVec_ Never-married', -1.3412453022191309),
 ('marital_statusclassVec_ Divorced', -0.6201596526565737),
 ('marital_statusclassVec_ Separated', -0.697281366471909),
 ('marital_statusclassVec_ Widowed', -0.2965429292865673),
 ('marital_statusclassVec_ Married-spouse-absent', -0.4970593499043775),
 ('occupationclassVec_ Prof-specialty', 0.24073239981525493),
 ('occupationclassVec_ Craft-repair', -0.1509133003060949),
 ('occupationclassVec_ Exec-managerial', 0.4532587098036229),
 ('occupationclassVec_ Adm-clerical', -0.23724640669791833),
 ('occupationclassVec_ Sales', 0.026823735617836315),
 ('occupationclassVec_ Other-service', -1.4167143979433718),
 ('occupationclassVec_ Machine-op-inspct', -0.5771454388958146),
 ('occupationclassVec_ ?', -0.6746848388464545),
 ('occupationclassVec_ Transport-moving', -0.4359212459330162),
 ('occupationclassVec_ Handlers-cleaners', -1.4479118249044114),
 ('occupationclassVec_ Farming-fishing', -1.2156381333487891),
 ('occupationclassVec_ Tech-support', 0.4539409331923003),
 ('occupationclassVec_ Protective-serv', 0.3871218365752616),
 ('occupationclassVec_ Priv-house-serv', -2.702160800337209),
 ('relationshipclassVec_ Husband', 0.3168753797100725),
 ('relationshipclassVec_ Not-in-family', -0.05864385404595789),
 ('relationshipclassVec_ Own-child', -1.7336731260219747),
 ('relationshipclassVec_ Unmarried', -0.6834366102324695),
 ('relationshipclassVec_ Wife', 1.3786503606200449),
 ('raceclassVec_ White', -0.7114074446167243),
 ('raceclassVec_ Black', -0.9565735780850516),
 ('raceclassVec_ Asian-Pac-Islander', -0.810280462722637),
 ('raceclassVec_ Amer-Indian-Eskimo', -1.540750013626492),
 ('sexclassVec_ Male', 0.5836316771832254),
 ('native_countryclassVec_ United-States', -1.0161288302187403),
 ('native_countryclassVec_ Mexico', -2.3764002993844553),
 ('native_countryclassVec_ ?', -1.5654510754387836),
 ('native_countryclassVec_ Philippines', -0.5732450077482386),
 ('native_countryclassVec_ Germany', -0.7176796299028012),
 ('native_countryclassVec_ Canada', -0.9171189775551005),
 ('native_countryclassVec_ Puerto-Rico', -1.7295030776155218),
 ('native_countryclassVec_ El-Salvador', -1.9348086265917555),
 ('native_countryclassVec_ India', -1.4370138737115754),
 ('native_countryclassVec_ Cuba', -0.7902546618977544),
 ('native_countryclassVec_ England', -1.1347311960964455),
 ('native_countryclassVec_ Jamaica', -0.8031071244943548),
 ('native_countryclassVec_ South', -2.006440699640529),
 ('native_countryclassVec_ China', -2.228387390424305),
 ('native_countryclassVec_ Italy', -0.36202851661852525),
 ('native_countryclassVec_ Dominican-Republic', -3.245136180463172

In [88]:
gbCvFeatureImportance = pd.DataFrame([(name, gbCvModel.bestModel.featureImportances[idx]) for idx, name in attrs],columns=['feature_name','feature_importance'])

print(gbCvFeatureImportance.sort_values(by=['feature_importance'],ascending =False))

feature_name feature_importance
23 marital_statusclassVec_ Married-civ-spouse 0.243193
96 education_num 0.148820
98 capital_loss 0.126782
99 hours_per_week 0.117401
97 capital_gain 0.107459
31 occupationclassVec_ Exec-managerial 0.089248
94 age 0.044000
1 workclassclassVec_ Self-emp-not-inc 0.031038
6 workclassclassVec_ Federal-gov 0.017678
29 occupationclassVec_ Prof-specialty 0.016123
34 occupationclassVec_ Other-service 0.011442
39 occupationclassVec_ Farming-fishing 0.011237
47 relationshipclassVec_ Wife 0.009036
52 sexclassVec_ Male 0.006487
17 educationclassVec_ Prof-school 0.003737
10 educationclassVec_ Bachelors 0.002830
30 occupationclassVec_ Craft-repair 0.002455
16 educationclassVec_ 7th-8th 0.001919
2 workclassclassVec_ Local-gov 0.001843
40 occupationclassVec_ Tech-support 0.001663
4 workclassclassVec_ State-gov 0.001284
59 native_countryclassVec_ Puerto-Rico 0.000892
9 educationclassVec_ Some-college 0.000878
11 educationclassVec_ Masters 0.000722
14 educationclassVec_ Assoc-acdm 0.000596
43 relationshipclassVec_ Husband 0.000592
95 fnlwgt 0.000512
41 occupationclassVec_ Protective-serv 0.000081
37 occupationclassVec_ Transport-moving 0.000051
73 native_countryclassVec_ Columbia 0.000000
.. ... ...
20 educationclassVec_ Doctorate 0.000000
18 educationclassVec_ 9th 0.000000
60 native_countryclassVec_ El-Salvador 0.000000
15 educationclassVec_ 10th 0.000000
13 educationclassVec_ 11th 0.000000
12 educationclassVec_ Assoc-voc 0.000000
8 educationclassVec_ HS-grad 0.000000
7 workclassclassVec_ Without-pay 0.000000
5 workclassclassVec_ Self-emp-inc 0.000000
3 workclassclassVec_ ? 0.000000
28 marital_statusclassVec_ Married-spouse-absent 0.000000
32 occupationclassVec_ Adm-clerical 0.000000
33 occupationclassVec_ Sales 0.000000
35 occupationclassVec_ Machine-op-inspct 0.000000
58 native_countryclassVec_ Canada 0.000000
57 native_countryclassVec_ Germany 0.000000
56 native_countryclassVec_ Philippines 0.000000
55 native_countryclassVec_ ? 0.000000
54 native_countryclassVec_ Mexico 0.000000
53 native_countryclassVec_ United-States 0.000000
51 raceclassVec_ Amer-Indian-Eskimo 0.000000
49 raceclassVec_ Black 0.000000
48 raceclassVec_ White 0.000000
46 relationshipclassVec_ Unmarried 0.000000
45 relationshipclassVec_ Own-child 0.000000
44 relationshipclassVec_ Not-in-family 0.000000
42 occupationclassVec_ Priv-house-serv 0.000000
38 occupationclassVec_ Handlers-cleaners 0.000000
36 occupationclassVec_ ? 0.000000
50 raceclassVec_ Asian-Pac-Islander 0.000000

[100 rows x 2 columns]

####<span style="color:darkblue">Question 9</span>
1. Build and train a RandomForestClassifier and print out a table of feature importances from it.

In [90]:
### Question 9.1 Answer ###

from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier()

rfModel = rf.fit(trainingData)

rfFeatureImportance =pd.DataFrame([(name, rfModel.featureImportances[idx]) for idx, name in attrs],columns=['feature_name','feature_importance'])


print(print(rfFeatureImportance.sort_values(by=['feature_importance'],ascending =False)))
  


feature_name feature_importance
23 marital_statusclassVec_ Married-civ-spouse 0.158841
97 capital_gain 0.148766
96 education_num 0.125487
43 relationshipclassVec_ Husband 0.109386
94 age 0.065538
99 hours_per_week 0.051287
44 relationshipclassVec_ Not-in-family 0.048412
10 educationclassVec_ Bachelors 0.038545
98 capital_loss 0.033300
45 relationshipclassVec_ Own-child 0.031277
52 sexclassVec_ Male 0.029699
31 occupationclassVec_ Exec-managerial 0.028662
24 marital_statusclassVec_ Never-married 0.020231
11 educationclassVec_ Masters 0.018170
17 educationclassVec_ Prof-school 0.016060
25 marital_statusclassVec_ Divorced 0.011761
47 relationshipclassVec_ Wife 0.009594
34 occupationclassVec_ Other-service 0.009387
29 occupationclassVec_ Prof-specialty 0.007462
8 educationclassVec_ HS-grad 0.006720
46 relationshipclassVec_ Unmarried 0.006621
20 educationclassVec_ Doctorate 0.004797
5 workclassclassVec_ Self-emp-inc 0.003576
30 occupationclassVec_ Craft-repair 0.002604
38 occupationclassVec_ Handlers-cleaners 0.001588
3 workclassclassVec_ ? 0.001331
35 occupationclassVec_ Machine-op-inspct 0.001294
6 workclassclassVec_ Federal-gov 0.000737
13 educationclassVec_ 11th 0.000698
95 fnlwgt 0.000665
.. ... ...
87 native_countryclassVec_ Laos 0.000000
83 native_countryclassVec_ Ireland 0.000000
86 native_countryclassVec_ Cambodia 0.000000
92 native_countryclassVec_ Hungary 0.000000
88 native_countryclassVec_ Thailand 0.000000
89 native_countryclassVec_ Yugoslavia 0.000000
90 native_countryclassVec_ Outlying-US(Guam-USVI-... 0.000000
91 native_countryclassVec_ Honduras 0.000000
93 native_countryclassVec_ Scotland 0.000000
7 workclassclassVec_ Without-pay 0.000000
82 native_countryclassVec_ Ecuador 0.000000
77 native_countryclassVec_ Portugal 0.000000
16 educationclassVec_ 7th-8th 0.000000
79 native_countryclassVec_ Peru 0.000000
59 native_countryclassVec_ Puerto-Rico 0.000000
61 native_countryclassVec_ India 0.000000
62 native_countryclassVec_ Cuba 0.000000
63 native_countryclassVec_ England 0.000000
27 marital_statusclassVec_ Widowed 0.000000
66 native_countryclassVec_ China 0.000000
22 educationclassVec_ 1st-4th 0.000000
70 native_countryclassVec_ Guatemala 0.000000
15 educationclassVec_ 10th 0.000000
72 native_countryclassVec_ Poland 0.000000
73 native_countryclassVec_ Columbia 0.000000
74 native_countryclassVec_ Taiwan 0.000000
75 native_countryclassVec_ Haiti 0.000000
76 native_countryclassVec_ Iran 0.000000
78 native_countryclassVec_ Nicaragua 0.000000
42 occupationclassVec_ Priv-house-serv 0.000000

[100 rows x 2 columns]
None